In [1]:
import cv2
import numpy as np

In [2]:
def list_available_cameras(max_index):
    """
    Lists the available camera indices.

    Args:
        max_index (int): The maximum index to check for available cameras.

    Returns:
        list: A list of indices where cameras are available.
    """

    # Store the indices of available cameras
    available_cameras = []

    # Iterate through the indices
    for i in range(max_index):
        cap = cv2.VideoCapture(i)

        # Check if the camera is available
        if cap.isOpened():
            # Add the index to the list of available cameras
            available_cameras.append(i)
            cap.release()        

    return available_cameras

max_camera_index = 10 # Maximum index to check for available cameras
available_cameras = list_available_cameras(max_camera_index) # Call the function

[ WARN:0@0.142] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@0.243] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@0.331] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@0.333] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@0.334] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@0.336] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@0.336] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@0.338] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@0.338] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@0.340] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup C

In [3]:
# Print the available cameras

print("Available Cameras:", available_cameras)

Available Cameras: [0, 2]


In [4]:
cap = cv2.VideoCapture(0)
img = cv2.imread('Putin.jpg')
